In [2]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

Setup Complete


In [8]:
train = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/train.csv")
sample_submission = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv")

In [4]:
file_dir = '../input/predict-volcanic-eruptions-ingv-oe/test'
!ls $file_dir | wc -l

4520


In [10]:
import glob
import os
import pandas as pd
files = glob.glob(os.path.join(file_dir, "*.csv"))
d1 = pd.read_csv(files[0])
d1.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
0,-511.0,NaN,-131.0,-457.0,47.0,-35.0,185.0,367.0,858.0,-492.0
1,-556.0,NaN,-105.0,-534.0,-7.0,-84.0,190.0,195.0,881.0,-368.0
2,-615.0,NaN,-97.0,-473.0,-50.0,8.0,219.0,327.0,937.0,-260.0
3,-682.0,NaN,-75.0,-388.0,-58.0,28.0,255.0,-249.0,995.0,-187.0
4,-763.0,NaN,-18.0,-358.0,-53.0,-104.0,271.0,-162.0,1032.0,-160.0


In [ ]:
def build_features(signal, ts, sensor_id):
    X = pd.DataFrame()
    f = np.fft.fft(signal)
    f_real = np.real(f)
    X.loc[ts, f'{sensor_id}_sum']       = signal.sum()
    X.loc[ts, f'{sensor_id}_mean']      = signal.mean()
    X.loc[ts, f'{sensor_id}_std']       = signal.std()
    X.loc[ts, f'{sensor_id}_var']       = signal.var() 
    X.loc[ts, f'{sensor_id}_max']       = signal.max()
    X.loc[ts, f'{sensor_id}_min']       = signal.min()
    X.loc[ts, f'{sensor_id}_skew']      = signal.skew()
    X.loc[ts, f'{sensor_id}_mad']       = signal.mad()
    X.loc[ts, f'{sensor_id}_kurtosis']  = signal.kurtosis()
    X.loc[ts, f'{sensor_id}_quantile99']= np.quantile(signal, 0.99)
    X.loc[ts, f'{sensor_id}_quantile95']= np.quantile(signal, 0.95)
    X.loc[ts, f'{sensor_id}_quantile85']= np.quantile(signal, 0.85)
    X.loc[ts, f'{sensor_id}_quantile75']= np.quantile(signal, 0.75)
    X.loc[ts, f'{sensor_id}_quantile55']= np.quantile(signal, 0.55)
    X.loc[ts, f'{sensor_id}_quantile45']= np.quantile(signal, 0.45) 
    X.loc[ts, f'{sensor_id}_quantile25']= np.quantile(signal, 0.25) 
    X.loc[ts, f'{sensor_id}_quantile15']= np.quantile(signal, 0.15) 
    X.loc[ts, f'{sensor_id}_quantile05']= np.quantile(signal, 0.05)
    X.loc[ts, f'{sensor_id}_quantile01']= np.quantile(signal, 0.01)
    X.loc[ts, f'{sensor_id}_fft_real_mean']= f_real.mean()
    X.loc[ts, f'{sensor_id}_fft_real_std'] = f_real.std()
    X.loc[ts, f'{sensor_id}_fft_real_max'] = f_real.max()
    X.loc[ts, f'{sensor_id}_fft_real_min'] = f_real.min()

    return X

0
500
1000


In [ ]:
train_set = list()
j=0
for seg in train.segment_id:
    signals = pd.read_csv(f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/{seg}.csv')
    train_row = []
    if j%500 == 0:
        print(j)
    for i in range(0, 10):
        sensor_id = f'sensor_{i+1}'
        train_row.append(build_features(signals[sensor_id].fillna(0), seg, sensor_id))
    train_row = pd.concat(train_row, axis=1)
    train_set.append(train_row)
    j+=1

train_set = pd.concat(train_set)

In [ ]:
train_set = train_set.reset_index()
train_set = train_set.rename(columns={'index': 'segment_id'})
train_set = pd.merge(train_set, train, on='segment_id')
train_set

In [ ]:
drop_cols = list()
for col in train_set.columns:
    if col == 'segment_id':
        continue
    if abs(train_set[col].corr(train_set['time_to_eruption'])) < 0.01:
        drop_cols.append(col)

In [ ]:
not_to_drop_cols = list()

for col1 in train_set.columns:
    for col2 in train_set.columns:
        if col1 == col2:
            continue
        if col1 == 'segment_id' or col2 == 'segment_id': 
            continue
        if col1 == 'time_to_eruption' or col2 == 'time_to_eruption':
            continue
        if abs(train_set[col1].corr(train_set[col2])) > 0.98:
            if col2 not in drop_cols and col1 not in not_to_drop_cols:
                drop_cols.append(col2)
                not_to_drop_cols.append(col1)
train = train_set.drop(['segment_id', 'time_to_eruption'], axis=1)
y = train_set['time_to_eruption']
reduced_y = y.copy()
reduced_train = train.copy()
reduced_train = reduced_train.drop(drop_cols, axis=1)
reduced_train

In [ ]:
train, val, y, y_val = train_test_split(train, y, random_state=666, test_size=0.2, shuffle=True)
reduced_train, reduced_val, reduced_y, reduced_y_val = train_test_split(reduced_train, reduced_y, random_state=666, test_size=0.2, shuffle=True)

# XGBoost!

In [ ]:
def test_models_cs(): 
    models_cs = dict
    for c in np.arrange(0.05, 0.1, 0.01)
        key = 
        models_cs[key] = xgb.XGBRFRegressor(n_estimators = 500,
                           learning_rate = 0.05, 
                           reg_lambda = 0.1,
                           eval_metric = mean_absolute_error, 
                           subsample = 0.9, 
                           colsample_bynode = c)
    return models_cs

def test_models_ne(): 
    models_ne = dict
    for n in arrange(100, 1000, 100)
        key = 
        models_ne[key] = xgb.XGBRFRegressor(n_estimators = n,
                           learning_rate = 0.05, 
                           reg_lambda = 0.1,
                           eval_metric = mean_absolute_error, 
                           subsample = 0.9, 
                           colsample_bynode = 0.1)
    return models_ne

def test_models_lr(): 
    models_lr = dict()
    for l in arrange(0.05, 0.2, 0.05)
        key = 
        models_lr[key] = xgb.XGBRFRegressor(n_estimators = 500,
                           learning_rate = l, 
                           reg_lambda = 0.1,
                           eval_metric = mean_absolute_error, 
                           subsample = 0.9, 
                           colsample_bynode = 0.1)
    return models_lr
    
def model_eval(model, X, y): 
    cross_val = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3)
    scores = cross_val_score(model, X, y, scoring = 'accuracy', cv = cross_val)
    return scores
    
models_cs = test_models_cs()
results_cs, names_cs = list(), list()
for name, model in models_cs.items(): 
    scores_cs = model_eval(model, reduced_train, reduced_y)
    results_cs.append(scores_cs)
    names_cs.append(name)

models_ne = test_models_ne()
results_ne, names_ne = list(), list()
for name, model in models_ne.items(): 
    scores_ne = model_eval(model, reduced_train, reduced_y)
    results_ne.append(scores_ne)
    names_ne.append(name)
    
models_lr = test_models_ne()
results_lr, names_lr = list(), list()
for name, model in models_lr.items(): 
    scores_lr = model_eval(model, reduced_train, reduced_y)
    results_lr.append(scores_lr)
    names_lr.append(name)

In [ ]:
print('>%s %.3f (%.3f)' % (name, mean(scores_cs), std(scores_cs)))
print('>%s %.3f (%.3f)' % (name, mean(scores_ne), std(scores_ne)))
print('>%s %.3f (%.3f)' % (name, mean(scores_lr), std(scores_lr)))

In [ ]:
fig = plt.figure(figsize = (8,8))
gs = fig.add_gridspec(2,2)

fig.add_suplot(gs[0,0])
sns.scatterplot(x = names_cs, y = results_cs)
sns.regplot(x = names_cs, y = results_cs)
plt.title("Scores by Colsample Value")

fig.add_suplot(gs[0,1])
sns.scatterplot(x = names_ne, y = results_ne)
sns.regplot(x = names_ne, y = results_ne)
plt.title("Scores by Num Estimators")

fig.add_suplot(gs[1,0])
sns.scatterplot(x = names_lr, y = results_lr)
sns.regplot(x = names_lr, y = results_lr)
plt.title("Scores by Learning Rate")